In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - examine_kinematics.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Look at Gaia-APOGEE data. Explore selections, metallicities, and output
selections to do mass modelling on.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np, pdb, sys, os, numbers, dill as pickle, copy
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib import path
import astropy.units as apu
# import kimmy

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA

sys.path.append('../../src/')
from ges_mass import plot as pplot
from ges_mass import util as putil

In [ ]:
# Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle')
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

# Colors setup
project_colors = pplot.colors()
rainbow_cmap = project_colors.colourmap('rainbow')

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','RO','VO','ZO']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ro,vo,zo\
    = putil.parse_config_dict(cdict,keywords)

# Directories
data_dir = data_dir_base+'gaia_apogee_processed/'
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
fig_dir = './fig/'

# Filenames
omask_kinematics_filename = data_dir+gaia_apogee_dir+'clean_kinematics.npy'

# Potential
mwpot = potential.MWPotential2014
potential.turn_physical_on(mwpot,ro=ro,vo=vo)
phi0 = potential.evaluatePotentials(mwpot,1e10,0).value

# Functions
def line_equation(x,m,b):
    return m*x+b
#def

## Get data

In [ ]:
# Cleaned kinematics
if os.path.exists(omask_kinematics_filename):
    print('Loading cleaned kinematics from '+omask_kinematics_filename)
    with open(omask_kinematics_filename,'rb') as f:
        omask_kinematics = pickle.load(f)
    ##pkl
else:
    sys.exit('Clean kinematics first')
##ie

gaia_omask,allstar_omask,os_omask,eELzs_omask,accs_omask,orbextr_omask = \
    omask_kinematics

# Unpack the orbital extrema
zmax_omask,rperi_omask,rapo_omask = orbextr_omask

In [ ]:
# Kinematic spaces information
kin_spaces = ['vRvT','Toomre','ELz','JRLz','eLz','AD']
n_kin_spaces = len(kin_spaces)

# Some keyword properties
AD_dedge = 1.2

# Limits dictionary. [ [xmin,xmax], [ymin,ymax] ]
lims_dict = {'vRvT':   [[-450,450], [-450,450]],
             'Toomre': [[-450,450], [0,450]],
             'ELz':    [[-4500,4500], [-2.,-0.]],
             'JRLz':   [[-4500,4500], [0,80]],
             'eLz':    [[-4500,4500], [-0.05,1.05]],
             'AD':     [[-AD_dedge,AD_dedge], [-AD_dedge,AD_dedge]]
            }

# Labels dictionary. [xlabel, ylabel]
labels_dict = {'vRvT':   [r'$v_{R}$',r'$v_{T}$'],
               'Toomre': [r'$v_{T}$',r'$(v_{R}^{2}+v_{z}^{2})^{1/2}$'],
               'ELz':    [r'$L_{z}$',r'$(E-\Phi_{0})/10^{5}$'],
               'JRLz':   [r'$L_{z}$',r'$\sqrt{J_{R}}$'],
               'eLz':    [r'$L_{z}$',r'eccentricity'],
               'AD':     [r'$L_{z}/J_{tot}$',r'$(J_{z}-J_{R})/J_\mathrm{tot}$']
               }

# Selection dictionaries [ [xcent,ycent], [xmajor,ymajor] ] or [ [x1,x2], [y1,y2] ]
# From Lane+ 2021
halo_selection_survey_dict = {'vRvT':   [ ['ellipse', [290,0], [110,35]], 
                                          ['ellipse', [-290,0], [110,35]] ],
                              'Toomre': [ ['ellipse', [0,300], [35,120]], ],
                              'ELz':    [ ['ellipse', [0,-1], [300,0.5]], ],
                              'JRLz':   [ ['ellipse', [0,45], [300,20]], ],
                              'eLz':    [ ['ellipse', [0,1], [500,0.025]], ],
                              'AD':     [ ['ellipse', [0,-1], [0.08,0.3]], ]
                              }
halo_selection_kws = {'linestyle':'dashed', 'linewidth':2.5, 'facecolor':'None', 
                      'edgecolor':'Black', 'color':'Black', 'label':r'High-$\beta$'}

# Misc
label_fontsize = 12
nbins = 40

In [ ]:
# Figure
fig = plt.figure()
fig.set_size_inches(10,5)
axs = fig.subplots(nrows=2, ncols=3).flatten()
for i in range( n_kin_spaces ):
    
    # Limits and labels
    xlim,ylim = lims_dict[kin_spaces[i]]
    xlabel,ylabel = labels_dict[kin_spaces[i]]
        
    # Purity
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    hist = np.histogram2d(xplot,yplot,bins=nbins,
                          range=[[xlim[0],xlim[1]],[ylim[0],ylim[1]]])[0]
    img = axs[i].imshow(np.log10(np.rot90(hist)), cmap=rainbow_cmap, vmin=-0.5, 
                        vmax=3.5, aspect='auto', 
                        extent=(xlim[0],xlim[1],ylim[0],ylim[1]))
    cax = fig.add_axes([0.92,0.15,0.025,0.7])
    cbar = fig.colorbar(img, cax=cax)
    cbar.ax.tick_params(labelsize=12) 
    cbar.set_label(r'$\log_{10} N$', fontsize=16)
    
    # Decoration
    pplot.axis_limits_and_labels(axs[i], xlim, ylim, xlabel, ylabel, 
        mixture_text=None, is_left_edge=True, is_top_edge=False, 
        label_fontsize=label_fontsize)
    if kin_spaces[i] == 'AD':
        pplot.add_diamond_boundary(axs[i], dedge=AD_dedge)
    ##fi
    pplot.add_selection_boundaries(axs[i], 
        halo_selection_survey_dict[kin_spaces[i]], 
        plot_kwargs=halo_selection_kws, plot_cent=False)
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.35, left=0.09, right=0.9)
fig.show()

## Examine different selections
### Create conditions to mask out disk contamination

In [ ]:
# Parameters for lines to mask GES
alpha_fe_m1, alpha_fe_b1 = -0.21,0.08
alpha_fe_m2, alpha_fe_b2 = -0.95,-0.58
mg_mn_m, mg_mn_b = 2.,0.55

# Mask GES with polygon
ges_polygon_pts = np.array([[-1.32,0.33],
                            [-0.92,0.24],
                            [-0.7,0.025],
                            [-0.88,-0.05],
                            [-1.1,0.1],
                            [-1.45,0.15]
                           ])

### Action Diamond

In [ ]:
selec_spaces = ['AD',]
n_selec_spaces = len(selec_spaces)
selec_spaces_suffix = '-'.join(selec_spaces)

In [ ]:
# Perform the selection on the kinematic spaces
for i in range(n_selec_spaces):
    
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), selec_spaces[i], phi0=phi0, 
        absolute=True)
    this_selection = halo_selection_survey_dict[selec_spaces[i]]
    
    # If this is a combined selection, handle multiple spaces
    if i == 0:
        combined_selec = pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    else:
        combined_selec = combined_selec & pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    ##ie
###i

# Get high-beta data
where_in_hb = np.where(combined_selec)[0]
gaia_hb = gaia_omask[where_in_hb]
allstar_hb = allstar_omask[where_in_hb]
os_hb = os_omask[where_in_hb]
eELzs_hb = eELzs_omask[:,where_in_hb]
accs_hb = accs_omask[:,where_in_hb]
orbextr_hb = orbextr_omask[:,where_in_hb]

# Create different selections to mask out disk stars to extract just GES stars. 
# So that os_hb[dmask...] are just GES stars
dmask_poly = path.Path(ges_polygon_pts).contains_points(
                        np.array([allstar_hb['FE_H'],allstar_hb['MG_FE']]).T)
dmask_line = ( allstar_hb['MG_FE']-allstar_hb['MN_FE']\
               > line_equation(allstar_hb['AL_FE'],mg_mn_m,mg_mn_b) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m2,alpha_fe_b2) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m1,alpha_fe_b1) )

# Assume dmask_poly is a subset of dmask_line. Get the stars within dmask_line 
# that are not in dmask_poly
dmask_inter = np.logical_and(dmask_line,np.logical_not(dmask_poly))

# Report number of stars in selections
print('Number of stars in selection is: '+str(len(where_in_hb)))
print('Number of GES stars within line disk mask is: '\
      +str(np.sum(dmask_line)))
print('Number of GES stars within polygon disk mask is: '\
      +str(np.sum(dmask_poly)))
print(str(np.sum(dmask_line)-np.sum(dmask_poly))+' GES stars in line disk mask'\
      ' not in polygon disk mask, len of dmask_inter: '+str(np.sum(dmask_inter)))

# Save the stellar selection for the lines and polygon approach

lines_id_dir = data_dir+gaia_apogee_dir+'hb_apogee_ids_'+selec_spaces_suffix\
                       +'_lines.npy'
poly_id_dir = data_dir+gaia_apogee_dir+'hb_apogee_ids_'+selec_spaces_suffix\
                       +'_poly.npy'
print('Saving line-selected APOGEE IDs to '+lines_id_dir)
print('Saving polygon-selected APOGEE IDs to '+poly_id_dir)
np.save(lines_id_dir,allstar_hb['APOGEE_ID'][dmask_line].astype(str))
np.save(poly_id_dir,allstar_hb['APOGEE_ID'][dmask_poly].astype(str))

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,5)
axs = fig.subplots(nrows=2, ncols=3).flatten()
for i in range( n_kin_spaces ):
    
    # Limits and labels
    xlim,ylim = lims_dict[kin_spaces[i]]
    xlabel,ylabel = labels_dict[kin_spaces[i]]
        
    # Purity
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    xplot_hb,yplot_hb = pplot.get_plottable_data( [os_hb,], [eELzs_hb,], 
        [accs_hb,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    hist = np.histogram2d(xplot,yplot,bins=nbins,
                          range=[[xlim[0],xlim[1]],[ylim[0],ylim[1]]])[0]
    img = axs[i].imshow(np.log10(np.rot90(hist)), cmap='Greys', vmin=-0.5, 
                        vmax=3.5, aspect='auto', 
                        extent=(xlim[0],xlim[1],ylim[0],ylim[1]))
    axs[i].scatter(xplot_hb, yplot_hb, s=10, facecolor='Red', edgecolor='None',
                   alpha=0.5, label='GE')
    
    # Decoration
    pplot.axis_limits_and_labels(axs[i], xlim, ylim, xlabel, ylabel, None,
        is_left_edge=True, is_top_edge=False, label_fontsize=18)
    if kin_spaces[i] == 'AD':
        pplot.add_diamond_boundary(axs[i], dedge=AD_dedge)
    ##fi
    
    if i == 5:
        halo_selection_kws_use = copy.deepcopy(halo_selection_kws)
        halo_selection_kws_use['edgecolor'] = 'Blue'
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws_use, plot_cent=False)
    else:
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws, plot_cent=False)
    ##ie
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.35, left=0.09, right=0.9)
fig.savefig(fig_dir+'kinematics_'+selec_spaces_suffix+'.png',dpi=300)
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
axs = fig.subplots(nrows=2, ncols=2).reshape(4)
n_bins = 50

abunds_x = ['FE_H','FE_H','FE_H','AL_FE']
abunds_y = ['MG_FE','CA_FE','C_N','MG_MN']
abunds_xlabels = ['[Fe/H]', '[Fe/H]', '[Fe/H]', '[Al/Fe]']
abunds_ylabels = ['[Mg/Fe]','[Ca/Fe]','[C/N]' , '[Mg/Mn]']
xlims = [ [-2.5,0.75], [-2.5,0.75], [-2.5,0.75], [-1.1,1.1] ]
ylims = [ [-0.3,0.7],  [-0.5,0.7],  [-1.8,1.1],  [-0.8,1.6] ]

for i in range(4):
    # Data
    xplot,xplot_err = putil.get_metallicity(allstar_hb,abunds_x[i])
    yplot,yplot_err = putil.get_metallicity(allstar_hb,abunds_y[i])
    # Background
    xplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_x[i])
    yplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_y[i])
    
    # Plot
    pts = axs[i].scatter(xplot, yplot, 
                         c=orbextr_hb[0], cmap=rainbow_cmap, 
                         edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
                         zorder=5)
    
#     pts = axs[i].scatter(xplot[dmask_line], yplot[dmask_line], 
#                          c=orbextr_hb[0][dmask_line], cmap=rainbow_cmap, 
#                          edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
#                          zorder=5)
#     pts = axs[i].scatter(xplot[~dmask_line], yplot[~dmask_line], 
#                          c=orbextr_hb[0][~dmask_line], cmap=rainbow_cmap, 
#                          edgecolors='Black', vmin=0, vmax=25, marker='s', s=16, 
#                          zorder=5)
#     axs[i].scatter(xplot[~dmask_line], yplot[~dmask_line], 
#                    c=orbextr_hb[0][~dmask_line], cmap=rainbow_cmap, 
#                    vmin=0, vmax=25, marker='o', s=16, zorder=5)
    hist,_,_ = np.histogram2d( xplot_bg, yplot_bg, bins=n_bins, 
                              range=[xlims[i],ylims[i]] )
    img = axs[i].imshow(np.rot90(np.log10(hist)), cmap='Greys', vmin=-1, vmax=5, 
                        extent=(xlims[i][0],xlims[i][1],ylims[i][0],ylims[i][1]), 
                        aspect='auto')
     
    cax = fig.add_axes([0.92,0.2,0.025,0.7])
    cbar = fig.colorbar(pts, cax=cax)
    cbar.ax.tick_params(labelsize=12) 
    cbar.set_label(r'z$_{max}$ [kpc]', fontsize=16)
    
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=20)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=20)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m1,alpha_fe_b1), 
                    linestyle='dashed', color='Black')
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m2,alpha_fe_b2), 
                    linestyle='dashed', color='Black')
        ssg_poly = patches.Polygon(ges_polygon_pts, closed=True, zorder=1, edgecolor='Black', facecolor='None')
        axs[i].add_artist(ssg_poly)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),mg_mn_m,mg_mn_b), 
                    linestyle='dashed', color='Black')
    
    # axs[i].grid(c='Black', zorder=3)
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.25, left=0.09, right=0.9)
fig.savefig(fig_dir+'abundances_'+selec_spaces_suffix+'.png',dpi=300)
fig.show()

### eccentricity - Lz

In [ ]:
selec_spaces = ['eLz',]
n_selec_spaces = len(selec_spaces)

In [ ]:
# Perform the selection on the kinematic spaces
for i in range(n_selec_spaces):
    
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), selec_spaces[i], phi0=phi0, 
        absolute=True)
    this_selection = halo_selection_survey_dict[selec_spaces[i]]
    
    # If this is a combined selection, handle multiple spaces
    if i == 0:
        combined_selec = pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    else:
        combined_selec = combined_selec & pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    ##ie
###i

# Get high-beta data
where_in_hb = np.where(combined_selec)[0]
gaia_hb = gaia_omask[where_in_hb]
allstar_hb = allstar_omask[where_in_hb]
os_hb = os_omask[where_in_hb]
eELzs_hb = eELzs_omask[:,where_in_hb]
accs_hb = accs_omask[:,where_in_hb]
orbextr_hb = orbextr_omask[:,where_in_hb]

# Create different selections to mask out disk stars to extract just GES stars. 
# So that os_hb[dmask...] are just GES stars
dmask_poly = path.Path(ges_polygon_pts).contains_points(
                        np.array([allstar_hb['FE_H'],allstar_hb['MG_FE']]).T)
dmask_line = ( allstar_hb['MG_FE']-allstar_hb['MN_FE']\
               > line_equation(allstar_hb['AL_FE'],mg_mn_m,mg_mn_b) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m2,alpha_fe_b2) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m1,alpha_fe_b1) )

# Assume dmask_poly is a subset of dmask_line. Get the stars within dmask_line 
# that are not in dmask_poly
dmask_inter = np.logical_and(dmask_line,np.logical_not(dmask_poly))

# Report number of stars in selections
print('Number of stars in selection is: '+str(len(where_in_hb)))
print('Number of GES stars within line disk mask is: '\
      +str(np.sum(dmask_line)))
print('Number of GES stars within polygon disk mask is: '\
      +str(np.sum(dmask_poly)))
print(str(np.sum(dmask_line)-np.sum(dmask_poly))+' GES stars in line disk mask'\
      ' not in polygon disk mask, len of dmask_inter: '+str(np.sum(dmask_inter)))

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,5)
axs = fig.subplots(nrows=2, ncols=3).flatten()
for i in range( n_kin_spaces ):
    
    # Limits and labels
    xlim,ylim = lims_dict[kin_spaces[i]]
    xlabel,ylabel = labels_dict[kin_spaces[i]]
        
    # Purity
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    xplot_hb,yplot_hb = pplot.get_plottable_data( [os_hb,], [eELzs_hb,], 
        [accs_hb,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    hist = np.histogram2d(xplot,yplot,bins=nbins,
                          range=[[xlim[0],xlim[1]],[ylim[0],ylim[1]]])[0]
    img = axs[i].imshow(np.log10(np.rot90(hist)), cmap='Greys', vmin=-0.5, 
                        vmax=3.5, aspect='auto', 
                        extent=(xlim[0],xlim[1],ylim[0],ylim[1]))
    axs[i].scatter(xplot_hb, yplot_hb, s=10, facecolor='Red', edgecolor='None',
                   alpha=0.5, label='GE')
    
    # Decoration
    pplot.axis_limits_and_labels(axs[i], xlim, ylim, xlabel, ylabel, None,
        is_left_edge=True, is_top_edge=False, label_fontsize=18)
    if kin_spaces[i] == 'AD':
        pplot.add_diamond_boundary(axs[i], dedge=AD_dedge)
    ##fi
    
    if i == 5:
        halo_selection_kws_use = copy.deepcopy(halo_selection_kws)
        halo_selection_kws_use['edgecolor'] = 'Blue'
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws_use, plot_cent=False)
    else:
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws, plot_cent=False)
    ##ie
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.35, left=0.09, right=0.9)
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
axs = fig.subplots(nrows=2, ncols=2).reshape(4)
n_bins = 50

abunds_x = ['FE_H','FE_H','FE_H','AL_FE']
abunds_y = ['MG_FE','CA_FE','C_N','MG_MN']
abunds_xlabels = ['[Fe/H]', '[Fe/H]', '[Fe/H]', '[Al/Fe]']
abunds_ylabels = ['[Mg/Fe]','[Ca/Fe]','[C/N]' , '[Mg/Mn]']
xlims = [ [-2.5,0.75], [-2.5,0.75], [-2.5,0.75], [-1.1,1.1] ]
ylims = [ [-0.3,0.7],  [-0.5,0.7],  [-1.8,1.1],  [-0.8,1.6] ]

for i in range(4):
    # Data
    xplot,xplot_err = putil.get_metallicity(allstar_hb,abunds_x[i])
    yplot,yplot_err = putil.get_metallicity(allstar_hb,abunds_y[i])
    # Background
    xplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_x[i])
    yplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_y[i])
    
    # Plot
    pts = axs[i].scatter(xplot, yplot, 
                         c=orbextr_hb[0], cmap=rainbow_cmap, 
                         edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
                         zorder=5)
    
#     pts = axs[i].scatter(xplot[dmask_poly], yplot[dmask_poly], 
#                          c=orbextr_hb[0][dmask_poly], cmap=rainbow_cmap, 
#                          edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
#                          zorder=5)
#     pts = axs[i].scatter(xplot[dmask_line], yplot[dmask_line], 
#                          c=orbextr_hb[0][dmask_line], cmap=rainbow_cmap, 
#                          edgecolors='Black', vmin=0, vmax=25, marker='s', s=16, 
#                          zorder=5)
#     axs[i].scatter(xplot[~dmask_line], yplot[~dmask_line], 
#                    c=orbextr_hb[0][~dmask_line], cmap=rainbow_cmap, 
#                    vmin=0, vmax=25, marker='o', s=16, zorder=5)
    hist,_,_ = np.histogram2d( xplot_bg, yplot_bg, bins=n_bins, 
                              range=[xlims[i],ylims[i]] )
    img = axs[i].imshow(np.rot90(np.log10(hist)), cmap='Greys', vmin=-1, vmax=5, 
                        extent=(xlims[i][0],xlims[i][1],ylims[i][0],ylims[i][1]), 
                        aspect='auto')
     
    cax = fig.add_axes([0.92,0.2,0.025,0.7])
    cbar = fig.colorbar(pts, cax=cax)
    cbar.ax.tick_params(labelsize=12) 
    cbar.set_label(r'z$_{max}$ [kpc]', fontsize=16)
    
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=20)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=20)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m1,alpha_fe_b1), 
                    linestyle='dashed', color='Black')
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m2,alpha_fe_b2), 
                    linestyle='dashed', color='Black')
        ssg_poly = patches.Polygon(ges_polygon_pts, closed=True, zorder=1, edgecolor='Black', facecolor='None')
        axs[i].add_artist(ssg_poly)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),mg_mn_m,mg_mn_b), 
                    linestyle='dashed', color='Black')
    
    # axs[i].grid(c='Black', zorder=3)
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.25, left=0.09, right=0.9)
fig.show()

### eLz and JRLz

In [ ]:
selec_spaces = ['eLz','JRLz']
n_selec_spaces = len(selec_spaces)

In [ ]:
n_selec_spaces

In [ ]:
# Perform the selection on the kinematic spaces
for i in range(n_selec_spaces):
    
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), selec_spaces[i], phi0=phi0, 
        absolute=True)
    this_selection = halo_selection_survey_dict[selec_spaces[i]]
    
    # If this is a combined selection, handle multiple spaces
    if i == 0:
        combined_selec = pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    else:
        combined_selec = combined_selec & pplot.is_in_scaled_selection(xplot, yplot, 
            this_selection, factor=[1.,1.])
    ##ie
###i

# Get high-beta data
where_in_hb = np.where(combined_selec)[0]
gaia_hb = gaia_omask[where_in_hb]
allstar_hb = allstar_omask[where_in_hb]
os_hb = os_omask[where_in_hb]
eELzs_hb = eELzs_omask[:,where_in_hb]
accs_hb = accs_omask[:,where_in_hb]
orbextr_hb = orbextr_omask[:,where_in_hb]

# Create different selections to mask out disk stars to extract just GES stars. 
# So that os_hb[dmask...] are just GES stars
dmask_poly = path.Path(ges_polygon_pts).contains_points(
                        np.array([allstar_hb['FE_H'],allstar_hb['MG_FE']]).T)
dmask_line = ( allstar_hb['MG_FE']-allstar_hb['MN_FE']\
               > line_equation(allstar_hb['AL_FE'],mg_mn_m,mg_mn_b) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m2,alpha_fe_b2) ) &\
             ( allstar_hb['MG_FE']\
               < line_equation(allstar_hb['FE_H'],alpha_fe_m1,alpha_fe_b1) )

# Assume dmask_poly is a subset of dmask_line. Get the stars within dmask_line 
# that are not in dmask_poly
dmask_inter = np.logical_and(dmask_line,np.logical_not(dmask_poly))

# Report number of stars in selections
print('Number of stars in selection is: '+str(len(where_in_hb)))
print('Number of GES stars within line disk mask is: '\
      +str(np.sum(dmask_line)))
print('Number of GES stars within polygon disk mask is: '\
      +str(np.sum(dmask_poly)))
print(str(np.sum(dmask_line)-np.sum(dmask_poly))+' GES stars in line disk mask'\
      ' not in polygon disk mask, len of dmask_inter: '+str(np.sum(dmask_inter)))

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,5)
axs = fig.subplots(nrows=2, ncols=3).flatten()
for i in range( n_kin_spaces ):
    
    # Limits and labels
    xlim,ylim = lims_dict[kin_spaces[i]]
    xlabel,ylabel = labels_dict[kin_spaces[i]]
        
    # Purity
    xplot,yplot = pplot.get_plottable_data( [os_omask,], [eELzs_omask,], 
        [accs_omask,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    xplot_hb,yplot_hb = pplot.get_plottable_data( [os_hb,], [eELzs_hb,], 
        [accs_hb,], np.array([1,]), kin_spaces[i], phi0=phi0, absolute=True)
    hist = np.histogram2d(xplot,yplot,bins=nbins,
                          range=[[xlim[0],xlim[1]],[ylim[0],ylim[1]]])[0]
    img = axs[i].imshow(np.log10(np.rot90(hist)), cmap='Greys', vmin=-0.5, 
                        vmax=3.5, aspect='auto', 
                        extent=(xlim[0],xlim[1],ylim[0],ylim[1]))
    axs[i].scatter(xplot_hb, yplot_hb, s=10, facecolor='Red', edgecolor='None',
                   alpha=0.5, label='GE')
    
    # Decoration
    pplot.axis_limits_and_labels(axs[i], xlim, ylim, xlabel, ylabel, None,
        is_left_edge=True, is_top_edge=False, label_fontsize=18)
    if kin_spaces[i] == 'AD':
        pplot.add_diamond_boundary(axs[i], dedge=AD_dedge)
    ##fi
    
    if i == 5:
        halo_selection_kws_use = copy.deepcopy(halo_selection_kws)
        halo_selection_kws_use['edgecolor'] = 'Blue'
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws_use, plot_cent=False)
    else:
        pplot.add_selection_boundaries(axs[i], 
            halo_selection_survey_dict[kin_spaces[i]],
            plot_kwargs=halo_selection_kws, plot_cent=False)
    ##ie
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.35, left=0.09, right=0.9)
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
axs = fig.subplots(nrows=2, ncols=2).reshape(4)
n_bins = 50

abunds_x = ['FE_H','FE_H','FE_H','AL_FE']
abunds_y = ['MG_FE','CA_FE','C_N','MG_MN']
abunds_xlabels = ['[Fe/H]', '[Fe/H]', '[Fe/H]', '[Al/Fe]']
abunds_ylabels = ['[Mg/Fe]','[Ca/Fe]','[C/N]' , '[Mg/Mn]']
xlims = [ [-2.5,0.75], [-2.5,0.75], [-2.5,0.75], [-1.1,1.1] ]
ylims = [ [-0.3,0.7],  [-0.5,0.7],  [-1.8,1.1],  [-0.8,1.6] ]

for i in range(4):
    # Data
    xplot,xplot_err = putil.get_metallicity(allstar_hb,abunds_x[i])
    yplot,yplot_err = putil.get_metallicity(allstar_hb,abunds_y[i])
    # Background
    xplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_x[i])
    yplot_bg,_ = putil.get_metallicity(allstar_omask,abunds_y[i])
    
    # Plot
    pts = axs[i].scatter(xplot, yplot, 
                         c=orbextr_hb[0], cmap=rainbow_cmap, 
                         edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
                         zorder=5)
    
#     pts = axs[i].scatter(xplot[dmask_poly], yplot[dmask_poly], 
#                          c=orbextr_hb[0][dmask_poly], cmap=rainbow_cmap, 
#                          edgecolors='None', vmin=0, vmax=25, marker='o', s=16, 
#                          zorder=5)
#     pts = axs[i].scatter(xplot[dmask_line], yplot[dmask_line], 
#                          c=orbextr_hb[0][dmask_line], cmap=rainbow_cmap, 
#                          edgecolors='Black', vmin=0, vmax=25, marker='s', s=16, 
#                          zorder=5)
#     axs[i].scatter(xplot[~dmask_line], yplot[~dmask_line], 
#                    c=orbextr_hb[0][~dmask_line], cmap=rainbow_cmap, 
#                    vmin=0, vmax=25, marker='o', s=16, zorder=5)
    hist,_,_ = np.histogram2d( xplot_bg, yplot_bg, bins=n_bins, 
                              range=[xlims[i],ylims[i]] )
    img = axs[i].imshow(np.rot90(np.log10(hist)), cmap='Greys', vmin=-1, vmax=5, 
                        extent=(xlims[i][0],xlims[i][1],ylims[i][0],ylims[i][1]), 
                        aspect='auto')
     
    cax = fig.add_axes([0.92,0.2,0.025,0.7])
    cbar = fig.colorbar(pts, cax=cax)
    cbar.ax.tick_params(labelsize=12) 
    cbar.set_label(r'z$_{max}$ [kpc]', fontsize=16)
    
    axs[i].set_xlabel(abunds_xlabels[i], fontsize=20)
    axs[i].set_ylabel(abunds_ylabels[i], fontsize=20)
    axs[i].set_xlim(xlims[i])
    axs[i].set_ylim(ylims[i])
    
    # Add lines
    if abunds_x[i] == 'FE_H' and abunds_y[i] == 'MG_FE':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m1,alpha_fe_b1), 
                    linestyle='dashed', color='Black', linewidth=0.5)
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),alpha_fe_m2,alpha_fe_b2), 
                    linestyle='dashed', color='Black', linewidth=0.5)
        ssg_poly = patches.Polygon(ges_polygon_pts, closed=True, zorder=1, 
            edgecolor='Black', facecolor='None', linewidth=0.5)
        axs[i].add_artist(ssg_poly)
    if abunds_x[i] == 'AL_FE' and abunds_y[i] == 'MG_MN':
        axs[i].plot(np.arange(-3,2,1.), line_equation(np.arange(-3,2,1.),mg_mn_m,mg_mn_b), 
                    linestyle='dashed', color='Black', linewidth=0.5)
    
    # axs[i].grid(c='Black', zorder=3)
###i

fig.tight_layout()
fig.subplots_adjust(hspace=0.35, wspace=0.25, left=0.09, right=0.9)
fig.show()